In [1]:
# Author: Aaron Marker (Randolph-Macon College)
# Python Script for posting tweets to twitter account

In [2]:
# import modules
import tweepy
import pandas as pd
import schedule
import time
import random
import datetime as dt
from datetime import timedelta, timezone
from statistics import median
import math

In [3]:
"""
    This is meant to be used with the GPT Script Provided
"""

'\n    This is meant to be used with the AI\n'

In [4]:
#Replace this with your own account's information and desired parameters
#This is the only block that needs to be customized
SCREEN_NAME = "CinderGameDev"
API_KEY = ""
API_KEY_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""
IMITATION_LIST_ID = "1545054152669007872"
TWEET_CSV_FILE_PATH = "C:/Downloads/TweetFile.csv"
TWEET_CSV_ROW_TO_START = 110
TIMES_TO_FOLLOW_A_DAY = 180
#Must each be less than or equal to 40 (unless you add more times below) (beware of max actions on twitter)
TIMES_TO_TWEET_A_DAY = 20 
TIMES_TO_RETWEET_A_DAY = 30 

In [5]:
#Set Up API
auth = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

thisUser = api.get_user(screen_name=SCREEN_NAME)
followerCount = thisUser.followers_count

In [6]:
# Provides a score of the prospective fruitfulness of following the account input
# userID: name of the account to score
# followsMe: whether or not the account follows you
# verbosity: how much information to print while scoring [0,2]
# returns the score for the account as a float
def followScorer(userID, followsMe = False, verbosity = 1):
    
    #Get Info
    userTweets = api.user_timeline(screen_name=userID,
                           count=200,
                           include_rts = False,
                           tweet_mode = 'extended'
                           )
    user = api.get_user(screen_name=userID)
    
    #Total Like and Retweet Score
    score = 0
    numOfPosts = 0
    numOfLikes = []
    numOfRetweets = []
    for tweet in userTweets:
        dateSincePosted = dt.datetime.now(timezone.utc) - tweet.created_at
        if(dateSincePosted.days > 60):
            break
        if tweet.in_reply_to_status_id is None:
            numOfPosts += 1
            numOfLikes.append(tweet.favorite_count)
            numOfRetweets.append(tweet.retweet_count)
    followersToFollowRatio = user.friends_count / (user.followers_count + .1)
    
    #Print Name Of account scoring
    if(verbosity > 0):
        print("Examining following potential of " + userID)
        
    #Like Score
    if(len(numOfLikes) == 0 or median(numOfLikes) == 0):
        if(verbosity > 1):
            print("\tlikes: 0")
    else:
        likeEquation = 2*(math.log(median(numOfLikes)) / math.log(user.followers_count/50))
        score += likeEquation
        if(verbosity > 1):
            print("\tlikes: " + str(likeEquation))
        
    #Retweet Score
    if(len(numOfRetweets) == 0 or median(numOfRetweets) == 0):
        if(verbosity > 1):
            print("\trts: 0")
    else:
        retweetEquation = 4*(math.log(median(numOfRetweets)) / math.log(user.followers_count/50))
        score += retweetEquation
        if(verbosity > 1):
            print("\trts: " + str(retweetEquation))
        
    #Follow Ratio Score
    if(verbosity > 1):
        print("\tfollowratio: " + str(10*math.log(2*followersToFollowRatio)))
    score += 10*math.log(2*followersToFollowRatio)
    
    #Posts Score
    if(numOfPosts == 0):
        if(verbosity > 1):
            print("\tPosts: 0")
    else:
        numOfPostEquation = math.log(numOfPosts)
        score += numOfPostEquation
        if(verbosity > 1):
            print("\tPosts: " + str(numOfPostEquation))
    
    #Followers Score
    followEquation = 5/(1+(1.05 ** (300 - user.followers_count)))
    if(verbosity > 1):
        print("\tFollowers: " + str(followEquation))
    score += followEquation
    
    #Follows Me Bonus Score
    if followsMe:
        score += 5
    
    #Minimum Recent Post Penalty
    if numOfPosts <= 3:
        score -= 100
    
    #Minimum Follower Penalty
    if user.followers_count <= 20:
        score -= 100
    
    #Print Final Score
    if(verbosity > 0):
        print("\tFinal Score: " + str(score))
        
    return score





# Provides a score of the quality of a tweet for retweeting
# tweet: the Id of the tweet to score
# verbosity: how much information to print while scoring [0,2]
# returns the score for the tweet as a float
def retweetScorer(tweet, verbosity = 1):
    
    score = 0
    
    #Print Account of Tweet Scoring
    if(verbosity > 0):
        print("Examining retweet potential of tweet by " + tweet.user.screen_name)
        
    #Account for Likes
    likeEquation = .05 * math.sqrt(tweet.favorite_count)
    score += likeEquation
    if(verbosity > 1):
        print("\tlike score:" + str(likeEquation))
        
    #Account for Retweets
    retweetEquation = 1 * math.sqrt(tweet.retweet_count)
    score += retweetEquation
    if(verbosity > 1):
        print("\tretweet score:" + str(retweetEquation))
        
    #Account for Follows
    followEquation = .0002 * math.sqrt(tweet.user.followers_count)
    score += followEquation
    if(verbosity > 1):
        print("\tfollow score:" + str(followEquation))
    
    #Print Final Score
    if(verbosity > 0):
        print("\tFinal Score: " + str(score))
        
    return score





# Takes the given user's most recent tweet and provides a retweet score
# userID: name of the account to score first tweet of
def scoreUserRecentTweet(userID):
    
    #Grab most recent tweets
    userTweets = api.user_timeline(screen_name=userID,
                               count=200,
                               include_rts = False,
                               tweet_mode = 'extended'
                               )
    
    #Go through tweets until one isn't a reply
    for tweet in userTweets:
        if tweet.in_reply_to_status_id is None:
            return retweetScorer(tweet)

        
        
        
        
        
"""
Examples to test
"""
# followScorer("CinderGameDev")
# followScorer("IAmSteveHarvey")
# scoreUserRecentTweet("CinderGameDev")
# scoreUserRecentTweet("MNateShyamalan")

'\nExamples to test\n'

In [7]:
def tweet():
    try:
        tweet = ""
        while len(str(tweet)) < 5:
            tweet = TweetData.iloc[0]
            TweetData.drop(index=TweetData.index[0], inplace=True)
        tweet = str(tweet.item())
        print("Stage 1: " + tweet + "\n\n")
        tweet = tweet[:280]
        print("Stage 1: " + tweet + "\n\n")
        tweet = tweet.replace("\\n", "\n")
        print("Stage 1: " + tweet + "\n\n")
        tweet = tweet.replace("0    ", "").strip()
        print("Stage 1: " + tweet + "\n\n")
        words = tweet.split()
        for word in words:
            if "http" in word:
                words.remove(word)
        tweet = " ".join(words)
        finalTweet = ""
        print("Stage 1: " + tweet + "\n\n")
        for pos, char in enumerate(tweet):
            if char == "." or char == "?" or char == "!":
                finalTweet = tweet[:pos]
        tweet = finalTweet.strip()
        api.update_status(tweet)
        print("Tweeted: " + tweet)
        print("(There are " + str(TweetData.shape[0]) + " Tweets remaining to be tweeted)")
    except:
        print("Error has occurred: There are most likely no tweets remaining to be tweeted or a tweet is in the incorrect format")
    
    
    
    
    
    
# Retweets a tweet from a user on the list specified
# (Can also retweet something they have retweeted)
def retweet():
    try:
        timeline = api.list_timeline(list_id=IMITATION_LIST_ID,include_rts=True,count=200)
        found = False
        for tweet in timeline:
            if tweet.retweeted == False and retweetScorer(tweet) > 13:
                api.retweet(tweet.id)
                found = True
                break;
        if(not found):
            print("Error: Failed to find Retweet-worthy content. Consider changing criteria in the retweetScorer() function to make retweeting easier")
        else:
            print("Retweeted: " + tweet.text)
    except:
        print("Error: Most likely retweeting too frequently. Skipping this Retweet round.")


        
        
        
        
# Follows a user from the list of those that follow the account or a list of those that
# follow the users in the imitation list
def follow():
    
    try:
        alreadySearched = [] # prevents retrying to follow the same account
        
        #Search your followers because someone new has followed you recently
        if(thisUser.followers_count > followerCount):
            print("Trying to select account to follow from your followers")
            for follower in api.followers(thisUser.screen_name):
                relationship = api.get_friendship(source_screen_name=thisUser.screen_name, target_screen_name=follower.screen_name)
                if follower.screen_name not in alreadySearched and not relationship[0].following and followScorer(follower.screen_name, followsMe = True) > 16:
                    api.create_friendship(screen_name=follower.screen_name)
                    print("Followed: " + follower.screen_name)
                    return
                else:
                    alreadySearched.append(follower.screen_name)
        
        #Search the followers of the accounts in the list
        tweetNotFound = True;
        while tweetNotFound:
            print("Trying to select account to follow from followers of those in list")
            tweets = api.list_timeline(list_id=IMITATION_LIST_ID,include_rts=True,count=100)
            for tweet in tweets:
                for follower in api.get_followers(screen_name=tweet.user.screen_name):
                    relationship = api.get_friendship(source_screen_name=thisUser.screen_name, target_screen_name=tweet.user.screen_name)
                    if follower.screen_name not in alreadySearched and not relationship[0].following and followScorer(follower.screen_name) > 16:
                        api.create_friendship(screen_name=follower.screen_name)
                        print("Followed: " + follower.screen_name)
                        return
                    else:
                        alreadySearched.append(follower.screen_name)

        print("Followed: " + user)
        
    except:
        print("Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier")
        
        
def loadTweets(verbosity = 1):
    pd.set_option("display.max_colwidth", 10000)
    df = pd.read_csv(TWEET_CSV_FILE_PATH, nrows=300, header=None)
    df.drop(df.head(TWEET_CSV_ROW_TO_START).index, inplace=True)
    if(verbosity > 1):
        print("Dropped first " + TWEET_CSV_ROW_TO_START + " rows")
    if(verbosity > 0):
        print("Loaded data with shape ", str(df.shape))
    return df

In [ ]:
#https://sproutsocial.com/insights/best-times-to-post-on-social-media/
bestPostTimes = ["09:0",
                "10:0",
                "10:3",
                "11:0",
                "08:0",
                "17:0",
                "12:0",
                "09:3",
                "11:3",
                "18:0",
                "08:3",
                "13:0",
                "12:3",
                "14:0",
                "19:0",
                "22:0",
                "03:0",
                "02:0",
                "15:0",
                "07:0",
                "19:3",
                "18:3",
                "05:0",
                "03:0",
                "21:0",
                "09:1",
                "10:1",
                "14:3",
                "09:4",
                "13:3",
                "17:3",
                "09:2",
                "10:1",
                "17:4",
                "18:4",
                "13:1",
                "14:1",
                "15:1",
                "12:1",
                "13:4",
                "09:5"]


TweetData = loadTweets()
    
bestTweetTimes = bestPostTimes[:TIMES_TO_TWEET_A_DAY]
for i in bestTweetTimes:
    schedule.every().day.at(i + str(random.randint(0,9))).do(tweet)
    
bestRetweetTimes = bestPostTimes[:TIMES_TO_RETWEET_A_DAY]
for i in bestRetweetTimes:
    schedule.every().day.at(i + str(random.randint(0,9))).do(retweet)

schedule.every(1440 / TIMES_TO_FOLLOW_A_DAY).minutes.do(follow)

print("TweetPusher Now Online.")

while True:
    schedule.run_pending()
    time.sleep(10)

Loaded data with shape  (112, 1)
TweetPusher Now Online.
Examining retweet potential of tweet by ADHDeanASL
	Final Score: 2.03131964239898
Examining retweet potential of tweet by davramdavram
	Final Score: 1.335653891793183
Examining retweet potential of tweet by JasonNotEvil
	Final Score: 2.0259337617788087
Examining retweet potential of tweet by JasonNotEvil
	Final Score: 1.7579845693476859
Examining retweet potential of tweet by JasonNotEvil
	Final Score: 1.4401473241519038
Examining retweet potential of tweet by davramdavram
	Final Score: 0.28565389179318296
Examining retweet potential of tweet by davramdavram
	Final Score: 1.201485131310953
Examining retweet potential of tweet by adamgreattweet
	Final Score: 3.6102756780897862
Examining retweet potential of tweet by JasonNotEvil
	Final Score: 6.350489082115568
Examining retweet potential of tweet by JasonNotEvil
	Final Score: 12.595738851755344
Examining retweet potential of tweet by JasonNotEvil
	Final Score: 10.70301201381012
Ex

Trying to select account to follow from followers of those in list
Examining following potential of TFBBroly
	Final Score: 17.87579641577409
Followed: TFBBroly
Trying to select account to follow from followers of those in list
Examining following potential of TexasBlueBloods
	Final Score: -72.93369420561856
Examining following potential of sandy_jane037
	Final Score: -64.15999501349276
Examining following potential of aaronhagen12
	Final Score: -71.20042134628282
Examining following potential of BrendanReichs
	Final Score: -4.35512392672954
Examining following potential of incognitoman924
	Final Score: -84.8844126111365
Examining following potential of Your_Mom_Always
	Final Score: -55.343180993475904
Examining following potential of beingMiaGoode
	Final Score: 0.7077945220712669
Examining following potential of GloverClaudio
	Final Score: 28.050367307038663
Followed: GloverClaudio
Trying to select account to follow from followers of those in list
Examining following potential of Parke

Examining following potential of RubenRa49199783
	Final Score: -177.41866189612801
Examining following potential of JDauwen
	Final Score: -74.59662643730434
Examining following potential of renshaw68
	Final Score: -148.44021794194273
Examining following potential of MF21227378
	Final Score: -156.40962652679818
Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier
Trying to select account to follow from followers of those in list
Examining following potential of supnabolde
	Final Score: -37.462791058095064
Examining following potential of ChristianStole5
	Final Score: 36.58237859770136
Followed: ChristianStole5
Trying to select account to follow from followers of those in list
Examining following potential of MaryToohey8
	Final Score: -134.48919445079449
Examining following potential of ethanrac
	Final Score: -71.03357291930823
Examining following potential of ericpaulm666
	Final Score: -55.958472857009994
Examining following potenti

Examining following potential of AbdoullahDumbu1
	Final Score: -62.15608005388617
Examining following potential of 1scd44
	Final Score: 42.18612112537806
Followed: 1scd44
Trying to select account to follow from followers of those in list
Examining following potential of christie5598
	Final Score: -90.40188447360478
Examining following potential of Hershy902
	Final Score: 15.88692061457142
Examining following potential of Brent0351
	Final Score: -65.01100379377009
Examining following potential of coastalbachelor
	Final Score: -77.57550128511924
Examining following potential of mcicatello
	Final Score: 23.5335794230181
Followed: mcicatello
Trying to select account to follow from followers of those in list
Examining following potential of AryasGranny
	Final Score: -75.87229178503432
Examining following potential of KellyElkin11
	Final Score: 17.91262115747015
Followed: KellyElkin11
Trying to select account to follow from followers of those in list
Examining following potential of ColinBr1

Examining following potential of jdouglas81
	Final Score: -151.77902418651308
Examining following potential of Alex_Las_Vegas
	Final Score: -71.47743433453486
Examining following potential of PatWallace
	Final Score: 17.19327958759377
Followed: PatWallace
Trying to select account to follow from followers of those in list
Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier
Trying to select account to follow from followers of those in list
Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier
Trying to select account to follow from followers of those in list
Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier
Trying to select account to follow from followers of those in list
Examining following potential of ethanlukegreen4
	Final Score: -57.63682257665432
Examining following potential of jdouglas81
	Final Score: -151.7790241

...

Trying to select account to follow from followers of those in list
Examining following potential of Kipalupagus
	Final Score: 12.66377670736519
Examining following potential of KhanKha95369671
	Final Score: -50.05647516938825
Examining following potential of yoldbag
	Final Score: 25.335402179022513
Followed: yoldbag
Trying to select account to follow from followers of those in list
Examining following potential of Kipalupagus
	Final Score: 12.66377670736519
Examining following potential of KhanKha95369671
	Final Score: -50.05647516938825
Examining following potential of yoldbag
	Final Score: 25.335402179022513
Followed: yoldbag
Trying to select account to follow from followers of those in list
Examining following potential of Kipalupagus
	Final Score: 12.66377670736519
Examining following potential of KhanKha95369671
	Final Score: -50.05647516938825
Examining following potential of yoldbag
	Final Score: 25.335402179022513
Followed: yoldbag
Trying to select account to follow from follow

Examining retweet potential of tweet by rickygervais
	Final Score: 4.1995091396411155
Examining retweet potential of tweet by JasonNotEvil
	Final Score: 16.60907698628166
Retweeted: RT @JasonNotEvil: Therapist: Do you have any hangups?

Me: https://t.co/vp97kiY2ef
Trying to select account to follow from followers of those in list
Examining following potential of Noreally3511817
	Final Score: 33.18009707314906
Followed: Noreally3511817
Trying to select account to follow from followers of those in list
Examining following potential of Noreally3511817
	Final Score: 33.18009707314906
Followed: Noreally3511817
Trying to select account to follow from followers of those in list
Examining following potential of Noreally3511817
	Final Score: 33.14080429175016
Followed: Noreally3511817
Trying to select account to follow from followers of those in list
Examining following potential of Noreally3511817
	Final Score: 33.14080429175016
Followed: Noreally3511817
Trying to select account to follow from

Followed: bigmant85186009
Trying to select account to follow from followers of those in list
Examining following potential of UsmanSaadat
	Final Score: -73.90717042578454
Examining following potential of suzwheel
	Final Score: -63.386562261503656
Examining following potential of nasirsa53610559
	Final Score: -73.68078913494006
Examining following potential of BillRambeth
	Final Score: -65.70287913721482
Examining following potential of bigmant85186009
	Final Score: 25.844265222952945
Followed: bigmant85186009
Trying to select account to follow from followers of those in list
Examining following potential of mentalibozuk
	Final Score: 17.850676296741142
Followed: mentalibozuk
Trying to select account to follow from followers of those in list
Examining following potential of mentalibozuk
	Final Score: 17.850095180125745
Followed: mentalibozuk
Trying to select account to follow from followers of those in list
Examining following potential of suzwheel
	Final Score: -63.386562261503656
Exam

Retweeted: RT @mxmclain: instead of flicking someone off just give them the thumbs down, it’s way more effective
Trying to select account to follow from followers of those in list
Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier
Trying to select account to follow from followers of those in list
Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier
Trying to select account to follow from followers of those in list
Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier
Trying to select account to follow from followers of those in list
Examining following potential of Xerxes6664
	Final Score: 27.699519882136535
Followed: Xerxes6664
Examining retweet potential of tweet by ShortSleeveSuit
	Final Score: 15.195848728557193
Retweeted: RT @OctopusCaveman: Why the fuck was I taught square dancing in school? I can barely read.
Error 

Retweeted: RT @eleniZarro: [Joining a gang]

me: so who do I stab for initiation?

members: again, this is a book club
Trying to select account to follow from followers of those in list
Examining following potential of James1213friday
	Final Score: -115.03009296024135
Examining following potential of stunnaqueenie_
	Final Score: -151.60140017165372
Error: Failed to follow. Consider changing criteria in the followScorer() function to make following easier
Trying to select account to follow from followers of those in list
Examining following potential of Alice99578890
	Final Score: -157.31301570669137
Examining following potential of jfbmcmlxxxv
	Final Score: -76.20676000392636
Examining following potential of JackGaughan22
	Final Score: -84.19164823777487
Examining following potential of thecinechick
	Final Score: 1.196130427281254
Examining following potential of Jesse_Kelsch
	Final Score: 24.85632725310707
Followed: Jesse_Kelsch
Trying to select account to follow from followers of tho